## Swedcham Debit Note Automation

1. Read American Credit Card Invoice
2. Gather all needed pieces of information
3. Create a new Word File with the current Debit Note
4. Generate a PDF from the Word File
5. Send a mail to Swedcham

#### 1. Read American Credit Card Invoice

In [23]:
# Import Libraries
import PyPDF2
from docxtpl import DocxTemplate
import datetime
import os
import re
import datetime
import locale
import json

# Email libraries
import smtplib, ssl
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [21]:
# Define Script Variables

# Get American Credit Card Invoice folder
cc_folder = 'D:\\Primeway\\Users Pw\\Oscar.Carvalho\\Contas\\Cartões de Crédito\\American\\'
cc_pass = 'LaylaLaylaLayla'

# Get Destination Folder and Template
nd_folder = 'D:\\Primeway\\Primeway\\Administração\\Contas a Receber\\Nota de Débito\\'
nd_template = 'Primeway - Nota de Débito - Template.docx'
nd_filename = "Primeway - Nota de Débito {} - {}-{}.docx"

# Charge parameters
nd_desc_template = "Plano Mensal MAILCHIMP - US$ {} (cotação R$ {} + 6,38% IOF + Comissão 10%)"
nd_IOF = 0.0638
nd_comission = 0.1

# Mail parameters
# mail_recipients = ['elaine@swedcham.com.br']
mail_recipients = ['oscar@primeway.com.br']
mail_sender = 'oscar@primeway.com.br'
smtp_success = 'Email enviado com sucesso!'
nd_mail_subject = 'Nota de Débito Mailchimp'
nd_mail_body = """
Olá Elaine,

Em anexo a Nota de Débito do Mailchimp deste mês.

Oscar
"""

In [ ]:
# Get last invoice file name

# Get current year
today = datetime.date.today()
year = str(today.year)
# print(year)

path = cc_folder + year + '\\'

file_list = [entry.name for entry in os.scandir(path) if entry.name[0:4] == year]
filename = file_list[len(file_list)-1]

print(path)
print(filename)

In [ ]:
# Open PDF file and get Mailchimp line
filename = '2023-07-Extrato-American-Bradesco.pdf'

with open(path + filename, 'rb') as obj:
   pdfReader = PyPDF2.PdfReader(obj)
   if pdfReader.is_encrypted:
      pdfReader.decrypt(cc_pass)
   page_txt = pdfReader.pages[0].extract_text()

pattern = '[0-9]+/[0-9]+/[0-9]+.+' + 'MAILCHIMP.+' + '\\n'
mailchimp_line = re.search(pattern, page_txt)
if (not mailchimp_line == None):
   mailchimp_line = mailchimp_line.group().replace(',', '.')
   print(mailchimp_line)
else:
   print('Mailchimp line not found!')

#### 2. Gather all needed pieces of information

In [ ]:
# Get most of the context variables from CC PDF

locale.setlocale(locale.LC_ALL, 'pt_BR')

items = re.split('[\s|\\n]', mailchimp_line)
nd_date_of_charge = items[0]
nd_value_usd      = items[4]
nd_value_brlusd   = items[5]
nd_value_brl      = items[6]

# Calc totals
nd_amount = round(float(nd_value_usd) * float(nd_value_brlusd) * (1 + nd_IOF) * (1 + nd_comission), 2)
nd_amount = str(nd_amount).replace('.', ',')

# Format values
nd_value_usd    = str(nd_value_usd).replace('.', ',')
nd_value_brlusd = str(round(float(nd_value_brlusd), 2)).replace('.', ',')

nd_description = nd_desc_template.format(nd_value_usd, nd_value_brlusd)
print(nd_description)

# Get current date
nd_current_date = datetime.date.today().strftime("%d de %B de %Y")
print(nd_current_date)


In [ ]:
# Get due date

# Get date object from date of charge
date_obj = datetime.datetime.strptime(nd_date_of_charge, '%d/%m/%Y')
print(date_obj)
# # Calc due date
nd_due_date = date_obj.replace(day=1) + datetime.timedelta(days=31*2)
nd_due_date = nd_due_date - datetime.timedelta(days=nd_due_date.day)

# Format Due Date
nd_due_date = nd_due_date.strftime("%d/%m/%Y")
print(nd_due_date)


In [ ]:
# Get sequence of the Nota de Débito

nd_sequence = ''

# Get filename
file_list = [entry.name for entry in os.scandir(nd_folder) if entry.name[-3:] == 'pdf']
filename = file_list[len(file_list)-1]
print(filename)

# Calc new sequence
pattern = '0[0-9]+(?=\s)'
last_sequence = re.search(pattern, filename)

nd_sequence = int(last_sequence.group()) + 1
nd_sequence = str(nd_sequence).zfill(4)
print(nd_sequence)

In [ ]:
# Prepare context

context = { 
   'nd_current_date'   : nd_current_date,
   'nd_sequence'       : nd_sequence,
   'nd_recipient'      : nd_recipient,
   'nd_description'    : nd_description,
   'nd_date_of_charge' : nd_date_of_charge,
   'nd_amount'         : nd_amount,
   'nd_due_date'       : nd_due_date
}
pretty_context = json.dumps(context, indent=4)

print(pretty_context)

#### 3. Create a new Word File from the current Debit Note

In [32]:
# Open template and render
doc = DocxTemplate(nd_folder + nd_template)
doc.render(context)

# Save a new Nota de Débito
nd_filename = nd_filename.format(nd_sequence, nd_due_date[-4:], nd_due_date[3:5])
doc.save(nd_folder + nd_filename)

#### 4. Generate a PDF from the Word File

In [36]:
# Create Word Application
from win32com import client
word_app = client.Dispatch("Word.Application")

# Generate PDF from Word
doc = word_app.Documents.Open(nd_folder + nd_filename)

nd_filename = nd_filename.replace('.docx', '.pdf')
doc.SaveAs(nd_folder + nd_filename, FileFormat=17)

#### 5. Send a mail with attached Debit Note to Swedcham

In [22]:
# Code References
   # https://www.youtube.com/watch?v=g_j6ILT-X0k&ab_channel=ThePyCoach
   # https://www.youtube.com/watch?v=NvtjLXSY-hE&ab_channel=LianneandJustin

# Google Resources
   # https://knowledge.workspace.google.com/kb/how-to-generate-an-app-passwords-000009237
   # https://app.s.center/hc/en-us/articles/1500012778262-Cannot-sign-in-to-Gmail-Invalid-email-address-or-password


# Google App Password: lpaf kvcc ytau ubvp


# Define email sender and receiver
# email_sender = 'write-email-here'
email_password = 'lpaf kvcc ytau ubvp'
# email_password = 'oscar'
# email_receiver = 'write-email-receiver-here'

# Mount the mail message
em = EmailMessage()
em['From'] = mail_sender
em['To'] = mail_recipients
em['Subject'] = nd_mail_subject
em.set_content(nd_mail_body)

# Add SSL (layer of security)
context = ssl.create_default_context()

# Log in and send the email
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(mail_sender, email_password)
    smtp_return = smtp.sendmail(mail_sender, mail_recipients, em.as_string())
    print(smtp_success)

Email enviado com sucesso!
